In [45]:
import pandas as pd

In [46]:
import csv
csv_col=[]
with open("data/input/ppp_loans_state_CO.csv") as fp:
    csv_reader=csv.reader(fp,delimiter=",")
    csv_col=next(iter(csv_reader))

In [47]:
import json
content=None
json_col=[]
with open("data/input/places.json") as fp:
    content=json.loads(fp.read())
    json_col = list(next(iter(content['data'])).keys())


In [48]:
common_values=list(set(csv_col).intersection(set(json_col)))
print(common_values)

['address']


In [49]:
places_df_audit_city=pd.DataFrame(content['data'])
loans_df_gov=pd.read_csv("data/input/ppp_loans_state_CO.csv")

In [50]:
loans_df_gov.columns,len(loans_df_gov.columns)

(Index(['loan_amount', 'business_name', 'address', 'city', 'state', 'zip',
        'naics_code', 'business_type', 'race_ethnicity', 'gender', 'veteran',
        'non_profit', 'jobs_reported', 'date_approved', 'lender', 'cd',
        'source_file', 'na_flag', 'dupe_flag', 'year_approved', 'address_clean',
        'state_clean', 'city_clean'],
       dtype='object'),
 23)

In [51]:
def get_st_road_data(address):
    if isinstance(address,dict):
        return address.get('street',None)
    return None
def get_city(address):
    if isinstance(address, dict):
        return address.get('city',None)
    return None
def get_state(address):
    if isinstance(address, dict):
        return address.get('adm1',None)
    return None
def get_zipcode(address):
    if isinstance(address, dict):
        return address.get('postcode',None)
    return None

In [52]:
places_df_audit_city["address"]
places_df_audit_city["address_clean"]=places_df_audit_city['address'].apply(get_st_road_data)
places_df_audit_city["city_clean"]=places_df_audit_city['address'].apply(get_city)
places_df_audit_city["state_clean"]=places_df_audit_city['address'].apply(get_state)
places_df_audit_city["zip"]=places_df_audit_city['address'].apply(get_zipcode)
places_df_audit_city["business_name"]=places_df_audit_city['name']

In [53]:
places_df_audit_city[['business_name','address_clean','city_clean','state_clean','zip']][:6]

,business_name,address_clean,city_clean,state_clean,zip
0,CBDermaceuticals,None,Denver,Colorado,80202
1,Unique Manufacturing & Marketing,5752 N Lamar St,Arvada,Colorado,80002
2,S N N Manufacturing Inc,None,Denver,Colorado,80209
3,Monarch Metal Manufacturing,550 W 53rd Pl,Denver,Colorado,80216
4,T & H Manufacturing LLC,581 S Federal Blvd,Denver,Colorado,80219
5,Sana Packaging,3451 Decatur St,Denver,Colorado,80211


In [54]:
import fuzzymatcher

In [55]:
on=["business_name","address_clean","city_clean","state_clean","zip"]

In [56]:
matched_results = fuzzymatcher.fuzzy_left_join(places_df_audit_city,
                                            loans_df_gov,
                                            on,
                                            on,)

In [57]:
final_result=matched_results[matched_results["best_match_score"]>0.8]

In [58]:
matched_results.columns

Index(['best_match_score', '__id_left', '__id_right', 'rank', 'name', 'logo',
       'description', 'address_left', 'location', 'reviews', 'category',
       'timezone', 'price_range', 'language', 'is_claimed', 'extra',
       'identity', 'opening_hours', 'status', 'media', 'activity', 'devices',
       'external_id', 'feature_id', 'gkg_id', 'owner', 'phone', 'url',
       'appointment', 'website', 'address_clean_left', 'city_clean_left',
       'state_clean_left', 'zip_left', 'business_name_left', 'loan_amount',
       'business_name_right', 'address_right', 'city', 'state', 'zip_right',
       'naics_code', 'business_type', 'race_ethnicity', 'gender', 'veteran',
       'non_profit', 'jobs_reported', 'date_approved', 'lender', 'cd',
       'source_file', 'na_flag', 'dupe_flag', 'year_approved',
       'address_clean_right', 'state_clean_right', 'city_clean_right'],
      dtype='object')

In [59]:
final_result=final_result.sort_values(by=['best_match_score'],ascending=False)

In [60]:
final_result.to_csv('data/output/matched_result.csv',index=True)

In [61]:
df1_cols=list(places_df_audit_city.columns)
on.append("address")
for i in range(len(df1_cols)):
    if df1_cols[i] in on:
        df1_cols[i]=f'{df1_cols[i]}_left'

In [62]:
len(df1_cols), len(places_df_audit_city.columns), df1_cols

(32,
 32,
 ['rank',
  'name',
  'logo',
  'description',
  'address_left',
  'location',
  'reviews',
  'category',
  'timezone',
  'price_range',
  'language',
  'is_claimed',
  'extra',
  'identity',
  'opening_hours',
  'status',
  'media',
  'activity',
  'devices',
  'external_id',
  'feature_id',
  'gkg_id',
  'owner',
  'phone',
  'url',
  'appointment',
  'website',
  'address_clean_left',
  'city_clean_left',
  'state_clean_left',
  'zip_left',
  'business_name_left'])

In [63]:
matched_df_1=final_result[df1_cols]

In [64]:
df2_cols=['loan_amount',
       'business_name_right', 'address_right', 'city', 'state', 'zip_right',
       'naics_code', 'business_type', 'race_ethnicity', 'gender', 'veteran',
       'non_profit', 'jobs_reported', 'date_approved', 'lender', 'cd',
       'source_file', 'na_flag', 'dupe_flag', 'year_approved',
       'address_clean_right', 'state_clean_right', 'city_clean_right']


In [65]:
len(loans_df_gov.columns),len(df2_cols)

(23, 23)

In [66]:
matched_df_2=final_result[df2_cols]

In [67]:
matched_df_1.columns

Index(['rank', 'name', 'logo', 'description', 'address_left', 'location',
       'reviews', 'category', 'timezone', 'price_range', 'language',
       'is_claimed', 'extra', 'identity', 'opening_hours', 'status', 'media',
       'activity', 'devices', 'external_id', 'feature_id', 'gkg_id', 'owner',
       'phone', 'url', 'appointment', 'website', 'address_clean_left',
       'city_clean_left', 'state_clean_left', 'zip_left',
       'business_name_left'],
      dtype='object')

In [68]:
refined_matched_df_1=matched_df_1[['rank', 'name', 'logo', 'description', 'address_left', 'location',
       'reviews', 'category', 'timezone', 'price_range', 'language',
       'is_claimed', 'extra', 'identity', 'opening_hours', 'status', 'media',
       'activity', 'devices', 'external_id', 'feature_id', 'gkg_id', 'owner',
       'phone', 'url', 'appointment', 'website',"address_clean_left","state_clean_left","city_clean_left","zip_left"]]

In [69]:
refined_matched_df_1.rename(columns={
    "name":"business_name",
    "address_left":"address",
    "address_clean_left":"address_clean",
    "state_clean_left":"state_clean",
    "city_clean_left":"clean",
    "zip_left":"zip"
})

,rank,business_name,logo,description,address,location,reviews,category,timezone,price_range,...,gkg_id,owner,phone,url,appointment,website,address_clean,state_clean,clean,zip
86477,96,"ABC Custom Iron & Locksmith, Inc",[https://lh6.googleusercontent.com/-qLWcD6Vjky...,None,"{'street': '550 Santa Fe Dr', 'postcode': '802...","{'lat': 39.724995, 'lng': -104.9983038}",None,"{'items': ['Door manufacturer', 'Fence contrac...",America/Denver,NaN,...,/g/11vzxtz9d4,"{'external_id': '112409930759442725854', 'url'...","[{'raw': '+1 303-534-0862', 'area_code': '303'...",https://www.google.com/maps/place/ABC+Custom+I...,None,"{'domain': 'abccustomiron.com', 'platform': No...",550 Santa Fe Dr,Colorado,Denver,80204
68909,78,Vexor Custom Woodworking Tools,[https://lh6.googleusercontent.com/-UYjqwTrwEV...,None,"{'street': '7162 Washington St', 'postcode': '...","{'lat': 39.826597, 'lng': -104.97640829999999}","{'rating': 4.5, 'count': 31, 'spread': [3, 0, ...","{'items': ['Tool manufacturer', 'Machine shop'...",America/Denver,0.0,...,/g/1tgpypnv,"{'external_id': '104453799335031209625', 'url'...","[{'raw': '+1 303-286-8906', 'area_code': '303'...",https://www.google.com/maps/place/Vexor+Custom...,None,"{'domain': 'vexorcwt.com', 'platform': None, '...",7162 Washington St,Colorado,Denver,80229
89347,98,The Infinite Monkey Theorem,[https://lh4.googleusercontent.com/-n0741ND_uO...,This bustling industrial-chic winery & taproom...,"{'street': '3200 Larimer St', 'postcode': '802...","{'lat': 39.764216399999995, 'lng': -104.9778219}","{'rating': 4.5, 'count': 357, 'spread': [17, 8...","{'items': ['Winery'], 'primary': None}",America/Denver,2.0,...,/g/1tk6wdg3,"{'external_id': '117708285165945337604', 'url'...","[{'raw': '+1 512-271-6807', 'area_code': '512'...",https://www.google.com/maps/place/The+Infinite...,None,"{'domain': 'theinfinitemonkeytheorem.com', 'pl...",3200 Larimer St,Colorado,Denver,80205
90034,99,"Golden West Plumbing, Heating, Air Conditionin...",[https://lh5.googleusercontent.com/-CQbpjmdcPB...,None,"{'street': '4217 Delaware St', 'postcode': '80...","{'lat': 39.7747321, 'lng': -104.99259939999999}","{'rating': 4.8, 'count': 2939, 'spread': [106,...","{'items': ['Plumber', 'Air conditioning contra...",America/Denver,0.0,...,/g/1tycx44f,"{'external_id': '107182873854475394021', 'url'...","[{'raw': '+1 720-303-3698', 'area_code': '720'...",https://www.google.com/maps/place/Golden+West+...,"{'platform': None, 'url': 'https://www.goldenw...","{'domain': 'goldenwestph.com', 'platform': Non...",4217 Delaware St,Colorado,Denver,80216
41265,52,Ralph's Industrial Sewing Machine Co,[https://lh4.googleusercontent.com/-crfKw5xdc_...,None,"{'street': '2030 Clay St', 'postcode': '80211'...","{'lat': 39.7481937, 'lng': -105.0202649}","{'rating': 4.5, 'count': 37, 'spread': [4, 0, ...","{'items': ['Industrial equipment supplier', 'S...",America/Denver,0.0,...,/g/1v1tnkvr,"{'external_id': '106151413625245673018', 'url'...","[{'raw': '+1 303-455-6831', 'area_code': '303'...",https://www.google.com/maps/place/Ralph%27s+In...,None,"{'domain': 'powersew.com', 'platform': None, '...",2030 Clay St,Colorado,Denver,80211
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56823,67,Hydro Rocky Mountain,[https://lh5.googleusercontent.com/-wKooAQiZqC...,None,"{'street': '115 W 56th Ave', 'postcode': '8021...","{'lat': 39.7986493, 'lng': -104.98911629999999}","{'rating': 5, 'count': 3, 'spread': [0, 0, 0, ...","{'items': ['Pump supplier', 'Engineering consu...",America/Denver,0.0,...,/g/1tgj99m5,"{'external_id': '117433601944834233244', 'url'...","[{'raw': '+1 303-293-8775', 'area_code': '303'...",https://www.google.com/maps/place/Hydro+Rocky+...,None,"{'domain': 'hydroinc.com', 'platform': None, '...",115 W 56th Ave,Colorado,Denver,80216
64927,73,Rio Grande Co Design Center,[https://lh4.googleusercontent.com/-Kv9YPIWP-1...,Local vendor supplying homeowners & developers...,"{'street': '123 Santa Fe Dr', 'postcode': '802..."

In [70]:
matched_df_2.columns

Index(['loan_amount', 'business_name_right', 'address_right', 'city', 'state',
       'zip_right', 'naics_code', 'business_type', 'race_ethnicity', 'gender',
       'veteran', 'non_profit', 'jobs_reported', 'date_approved', 'lender',
       'cd', 'source_file', 'na_flag', 'dupe_flag', 'year_approved',
       'address_clean_right', 'state_clean_right', 'city_clean_right'],
      dtype='object')

In [81]:
matched_df_2.rename(columns={
    "business_name_right":"business_name",
    "address_right":"address",
    "address_clean_right":"address_clean",
    "state_clean_right":"state_clean",
    "city_clean_right":"city_clean",
    "zip_right":"zip_hello, world"
})

,loan_amount,business_name,address,city,state,"zip_hello, world",naics_code,business_type,race_ethnicity,gender,...,date_approved,lender,cd,source_file,na_flag,dupe_flag,year_approved,address_clean,state_clean,city_clean
86477,40700.0,"ABC CUSTOM IRON & LOCKSMITH, INC.",550 Santa Fe Drive,Denver,CO,80204.0,238990.0,Corporation,Unanswered,Unanswered,...,2020-04-28,"BOKF, National Association",CO-01,01 PPP sub 150k through 112420.csv,False,False,2020,550 SANTA FE DR,CO,DENVER
68909,161437.0,VEXOR CUSTOM WOODWORKING TOOLS INC.,7162 WASHINGTON ST,DENVER,CO,80229.0,332710.0,Corporation,Unanswered,Unanswered,...,2020-05-01,"U.S. Bank, National Association",CO-07,150k plus PPP through 112420.csv,False,False,2020,7162 WASHINGTON ST,CO,DENVER
89347,166500.0,THE INFINITE MONKEY THEOREM,3200 Larimer Street,Denver,CO,80205.0,312130.0,Corporation,Unanswered,Unanswered,...,2020-04-11,MidFirst Bank,CO-01,150k plus PPP through 112420.csv,False,False,2020,3200 LARIMER ST,CO,DENVER
90034,392898.0,"GOLDEN WEST PLUMBING, HEATING & AIR CONDITIONI...",4217 Delaware St,Denver,CO,80216.0,811412.0,Subchapter S Corporation,Unanswered,Unanswered,...,2020-04-30,Bank of the West,CO-01,150k plus PPP through 112420.csv,False,False,2020,4217 DELAWARE ST,CO,DENVER
41265,127500.0,RALPH'S INDUSTRIAL SEWING MACHINE COMPANY,2030 Clay street,DENVER,CO,80211.0,423830.0,Corporation,Hispanic,Male Owned,...,2020-04-13,KeyBank National Association,CO-01,01 PPP sub 150k through 112420.csv,False,False,2020,2030 CLAY ST,CO,DENVER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56823,309157.5,"HYDRO ROCKY MOUNTAIN, INC.",115 56TH AVE,DENVER,CO,80216.0,333914.0,Corporation,Unanswered,Male Owned,...,2020-04-15,Fifth Third Bank,CO-01,150k plus PPP through 112420.csv,False,False,2020,115 56 TH AVE,CO,DENVER
64927,2776513.0,RIO GRANDE CO.,201 SANTA FE DRIVE,DENVER,CO,80223.0,423310.0,Subchapter S Corporation,Unanswered,Unanswered,...,2020-04-11,KeyBank National Association,CO-01,150k plus PPP through 112420.csv,False,False,2020,201 SANTA FE DR,CO,DENVER
85396,2205500.0,SEATTLE FISH COMPANY,6211 E. 42nd Ave.,DENVER,CO,80216.0,424410.0,Corporation,Unanswered,Unanswered,...,2020-04-08,"UMB Bank, National Association",CO-01,150k plus PPP through 112420.csv,False,False,2020,6211 E 42 ND AVE,CO,DENVER
17819,353262.0,THE LAKEWOOD BRICK AND TILE COMPANY,1325 JAY ST,LAKEWOOD,CO,80214.0,327120.0,Corporation,Unanswered,Unanswered,...,2020-04-11,Fowler State Bank,CO-07,150k plus PPP through 112420.csv,False,False,2020,1325 JAY ST,CO,LAKEWOOD


In [83]:
matched_df_2.columns,len(matched_df_2.columns)

(Index(['loan_amount', 'business_name_right', 'address_right', 'city', 'state',
        'zip_right', 'naics_code', 'business_type', 'race_ethnicity', 'gender',
        'veteran', 'non_profit', 'jobs_reported', 'date_approved', 'lender',
        'cd', 'source_file', 'na_flag', 'dupe_flag', 'year_approved',
        'address_clean_right', 'state_clean_right', 'city_clean_right'],
       dtype='object'),
 23)

In [77]:
refined_matched_df_1.to_csv("data/output/matched_df_1.csv")

In [74]:
matched_df_2.to_csv("data/output/matched_df_2.csv")